In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import LightGBMModel
import lightgbm as lgb

In [ ]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import RNNModel
from darts.dataprocessing.transformers import Scaler
import warnings
import torch

warnings.filterwarnings("ignore")

# Set default tensor dtype to float32
torch.set_default_dtype(torch.float32)

# Check if MPS is available and set device accordingly
device = 'cpu'
if torch.backends.mps.is_available():
    device = 'mps'

# Load ais_train.csv with separator '|'
train_df = pd.read_csv('ais_train.csv', sep='|')
train_df['time'] = pd.to_datetime(train_df['time'])

# Load ais_test.csv with separator ','
test_df = pd.read_csv('ais_test.csv', sep=',')
test_df['time'] = pd.to_datetime(test_df['time'])

# Use 'vesselId' instead of 'vessel_id'
# Select only vessel IDs that are in both train and test datasets
common_vessel_ids = set(train_df['vesselId']).intersection(set(test_df['vesselId']))
train_df = train_df[train_df['vesselId'].isin(common_vessel_ids)]

# Additional features to include
feature_columns = ['latitude', 'longitude', 'cog', 'sog']  # Add other relevant features if available

# Preprocess the data
timeseries_dict = {}
last_train_time = {}
scaler_dict = {}

# Process each vesselId group
for vessel_id, group_df in train_df.groupby('vesselId'):
    # Sort on time
    group_df = group_df.sort_values('time')
    # Set index to time
    group_df = group_df.set_index('time')
    # Select features
    features_df = group_df[feature_columns]
    # Resample data to 30-minute frequency with mean and interpolate
    features_df = features_df.resample('30T').mean().interpolate(method='linear')
    # Ensure data is in float32
    features_df = features_df.astype(np.float32)
    # Create TimeSeries object without 'dtype' parameter
    ts = TimeSeries.from_dataframe(features_df)
    # Convert TimeSeries data to float32 using astype()
    ts = ts.astype(np.float32)
    # Scaling the data
    scaler = Scaler()
    ts_scaled = scaler.fit_transform(ts)
    # Store the TimeSeries object and last training time
    timeseries_dict[vessel_id] = ts_scaled
    last_train_time[vessel_id] = features_df.index.max()
    scaler_dict[vessel_id] = scaler

# Initialize a dictionary to store predictions
predictions = {}

# Fit RNN models and predict for each TimeSeries object
for vessel_id, ts_scaled in timeseries_dict.items():
    # Get the last training time
    last_time = last_train_time[vessel_id]
    # Get test times for this vessel
    vessel_test_df = test_df[test_df['vesselId'] == vessel_id]
    test_times = vessel_test_df['time']
    if test_times.empty:
        continue  # Skip if no test times for this vessel
    # Compute the time differences in 30-minute intervals
    time_diffs = (test_times - last_time).dt.total_seconds() / (30 * 60)
    # Get the maximum forecast horizon needed
    max_n = int(np.ceil(time_diffs.max()))
    if max_n <= 0:
        continue  # Skip if no future times to predict
    # Initialize RNN model with hyperparameter tuning
    model = RNNModel(
        model='LSTM',
        hidden_dim=50,
        dropout=0.2,
        batch_size=16,
        n_epochs=200,
        optimizer_kwargs={'lr': 1e-3},
        model_name=f'RNNModel_{vessel_id}',
        log_tensorboard=False,  # Set to False if not using TensorBoard
        random_state=42,
        input_chunk_length=24,
        output_chunk_length=12,
        likelihood=None,
        # Ensure the model uses the correct device
        pl_trainer_kwargs={
            "accelerator": device,
            "devices": 1
        }
    )
    # Fit the model
    model.fit(ts_scaled)
    # Predict up to the maximum horizon needed
    forecast_scaled = model.predict(n=max_n)
    # Inverse transform the forecast
    forecast = scaler_dict[vessel_id].inverse_transform(forecast_scaled)
    # Store the forecast and last time
    predictions[vessel_id] = (forecast, last_time)

# Initialize a list to store submission rows
submission_rows = []

# Generate predictions for the submission file
for idx, row in test_df.iterrows():
    vessel_id = row['vesselId']
    test_time = row['time']
    test_id = row['ID']  # Assuming 'ID' column exists in test_df
    # Check if predictions are available for this vessel_id
    if vessel_id in predictions:
        forecast_ts, last_time = predictions[vessel_id]
        time_diff = (test_time - last_time).total_seconds() / (30 * 60)
        index = int(np.round(time_diff)) - 1  # Adjust index
        # Check if index is within forecast horizon
        if 0 <= index < len(forecast_ts):
            predicted_lat = forecast_ts['latitude'].values()[index][0]
            predicted_lon = forecast_ts['longitude'].values()[index][0]
        else:
            predicted_lat = np.nan
            predicted_lon = np.nan
    else:
        predicted_lat = np.nan
        predicted_lon = np.nan
    # Append the prediction to the submission list
    submission_rows.append({
        'ID': test_id,
        'longitude_predicted': predicted_lon,
        'latitude_predicted': predicted_lat
    })

# Create a submission DataFrame from the list
submission_df = pd.DataFrame(submission_rows)

# Save the submission file
submission_df.to_csv('submission.csv', index=False)


ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 11.2 K | train
6 | V               | Linear           | 204    | train
-------------------------------------------------------------
11.4 K    Trainable params
0         Non-trainable params
11.4 K    Total params
0.046     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Training: |                                                                                                   …

In [ ]:

# Initialize a list to store submission rows
submission_rows = []

# Generate predictions for the submission file
for idx, row in test_df.iterrows():
    vessel_id = row['vesselId']
    test_time = row['time']
    test_id = row['ID']  # Assuming 'ID' column exists in test_df
    # Check if predictions are available for this vessel_id
    if vessel_id in predictions:
        forecast_ts, last_time = predictions[vessel_id]
        time_diff = (test_time - last_time).total_seconds() / (30 * 60)
        index = int(np.round(time_diff)) - 1  # Adjust index
        # Check if index is within forecast horizon
        if 0 <= index < len(forecast_ts):
            predicted_lat = forecast_ts['latitude'].values()[index][0]
            predicted_lon = forecast_ts['longitude'].values()[index][0]
        else:
            predicted_lat = np.nan
            predicted_lon = np.nan
    else:
        predicted_lat = np.nan
        predicted_lon = np.nan
    # Append the prediction to the submission list
    submission_rows.append({
        'ID': test_id,
        'longitude_predicted': predicted_lon,
        'latitude_predicted': predicted_lat
    })

# Create a submission DataFrame from the list
submission_df = pd.DataFrame(submission_rows)

# Save the submission file
submission_df.to_csv('submission.csv', index=False)
